In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime
from typing import Tuple


# Section 1: Setup


## Load Tickers

In [37]:
tickers = ['AAPL', 'MSFT', 'GOOGL']
years = 5


start_date = datetime.now() - pd.Timedelta(days=365*years)
end_date = datetime.now()

In [38]:
hist_data = pd.DataFrame()

for ticker in tickers:
    hist_data[ticker] = yf.download(ticker, start="2020-01-01", end="2024-12-01")['Adj Close']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [39]:
display(hist_data.head())

,AAPL,MSFT,GOOGL
Date,,,
2020-01-02,72.796028,153.630707,68.264961
2020-01-03,72.088272,151.717712,67.907845
2020-01-06,72.662704,152.109879,69.717865
2020-01-07,72.320976,150.722977,69.583206
2020-01-08,73.484344,153.123764,70.078476


## Calculate Log Returns

In [40]:
returns = np.log(hist_data / hist_data.shift(1))
returns.dropna(inplace=True)

display(returns.head())

,AAPL,MSFT,GOOGL
Date,,,
2020-01-03,-0.009770,-0.012530,-0.005245
2020-01-06,0.007937,0.002582,0.026305
2020-01-07,-0.004714,-0.009160,-0.001933
2020-01-08,0.015958,0.015803,0.007092
2020-01-09,0.021018,0.012416,0.010443


# Find Covariance Matrix

In [41]:
cov_matrix = returns.cov() * 252
display(cov_matrix)

,AAPL,MSFT,GOOGL
AAPL,0.101462,0.073438,0.067976
MSFT,0.073438,0.094101,0.074917
GOOGL,0.067976,0.074917,0.105434


In [42]:
expected_returns = pd.DataFrame(returns.mean() * 252, columns=['Annual Return'])
display(expected_returns)

,Annual Return
AAPL,0.240948
MSFT,0.206719
GOOGL,0.184760


In [50]:
def calculate_portfolio_metrics(weights: np.ndarray, returns: np.ndarray, cov_matrix: np.ndarray) -> Tuple[float, float]:
    #returns portofolio return and volatility as a tuple
    portfolio_return = np.sum(returns * weights)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return portfolio_return, portfolio_volatility



In [53]:
 # intialize weights
num_assets = len(tickers)
init_weights = np.array([1.0/num_assets] * num_assets)

calculate_portfolio_metrics(weights=init_weights, returns=expected_returns["Annual Return"], cov_matrix=cov_matrix)

(np.float64(0.21080898841670537), np.float64(0.285513239643252))